In [15]:
import pandas as pd

In [16]:
from py2neo import Graph, authenticate, Relationship

In [17]:
graph = Graph(user="neo4j", password="admin")

In [18]:
#erase db
graph.run('''
MATCH (n)
DETACH DELETE n
''')

In [19]:
df = pd.DataFrame.from_csv("C:\\Users\\thaunayf\\Documents\Neo4j\\DC10_3\\import\\day1.csv", index_col=None)
accountsOrig = list(set(df["fakeNameOrig"]))
accountsDest = list(set(df["fakeNameDest"]))
accounts = list(set(accountsOrig + accountsDest))

In [20]:
for i,row in enumerate(accounts):
    graph.run('''
    
   CREATE (a:Account{Name:$name}) 
   
    ''',{'name':row})
    
    
x = graph.run('''
        MATCH (a:Account)
        RETURN COUNT(a) AS ACCOUNT
    ''').data()[0]

print("Writing %d accounts" % x['ACCOUNT'])

Writing 1800 accounts


# Loading from dataFrame ...

In [31]:
df = pd.DataFrame.from_csv("C:\\Users\\thaunayf\\Documents\Neo4j\\DC10_3\\import\\day1.csv", index_col=None)

In [32]:
df.head()

,Unnamed: 0,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,typeAsNumeric,origIsCustomer,destIsCustomer,fakeNameOrig,fakeNameDest
0,0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,2,1,0,C123,M197
1,1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,2,1,0,C166,M204
2,2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,1,1,1,C130,C553
3,3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,0,1,1,C840,C389
4,4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,2,1,0,C204,M123


In [36]:
df = pd.DataFrame.from_csv("C:\\Users\\thaunayf\\Documents\Neo4j\\DC10_3\\import\\hour"+str(1)+".csv")

In [37]:
df.head()

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud,typeAsNumeric,origIsCustomer,destIsCustomer,fakeNameOrig,fakeNameDest
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,2,1,0,C123,M197
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,2,1,0,C166,M204
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,1,1,1,C130,C553
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,0,1,1,C840,C389
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,2,1,0,C204,M123


In [39]:
for j in range(1,24):
    df = pd.DataFrame.from_csv("C:\\Users\\thaunayf\\Documents\Neo4j\\DC10_3\\import\\hour"+str(j)+".csv")

    for i,row in df.iterrows():
        graph.run('''
    
    MATCH (a:Account{Name:$nameOrig})
    MATCH (b:Account{Name:$nameDest})
    
    CREATE (a)-[action:RELTYPE{type:$type, step:$step, amount:$amount, oldBalanceOrig:$oldBalanceOrig, 
    newBalanceOrig:$newBalanceOrig, oldBalanceDest:$oldBalanceDest, newBalanceDest:$newBalanceDest, dataLine:$i}]->(b)
    
    
    ''',{'step':row.step, 'type':row.type, 'amount':row.amount, 'nameOrig':row.fakeNameOrig, 'oldBalanceOrig':row.oldBalanceOrig,
         'newBalanceOrig':row.newBalanceOrig, 'nameDest':row.fakeNameDest, 'oldBalanceDest':row.oldBalanceDest,
              'newBalanceDest':row.newBalanceDest, 'i':int(i)})
            
    x = graph.run('''
        MATCH ()-[r]->()
        RETURN COUNT(r) AS OPERATION
    ''').data()[0]

    print("Writing %d operations" % x['OPERATION'])

    

Writing 2708 operations
Writing 3722 operations
Writing 4274 operations
Writing 4839 operations
Writing 5504 operations
Writing 7164 operations
Writing 14001 operations
Writing 35098 operations
Writing 72726 operations
Writing 108717 operations
Writing 145958 operations
Writing 182111 operations
Writing 219626 operations
Writing 261111 operations
Writing 305720 operations
Writing 348191 operations
Writing 391552 operations
Writing 441131 operations
Writing 492483 operations
Writing 533108 operations
Writing 552260 operations
Writing 564895 operations
Writing 571039 operations


# or from CSV ...

In [ ]:
for i in range(744):
    graph.run('''
    
    USING PERIODIC COMMIT 100 LOAD CSV FROM "file:///hour'''+str(i)+'''.csv"  AS line

    MERGE (a:Account{Name:line[4]})
    MERGE (b:Account{Name:line[7]})
    CREATE (a)-[action:RELTYPE{type:line[2], step:line[1], amount:line[3], oldbalanceOrig:line[5], 
    newbalanceOrig:line[6], oldbalanceDest:line[8], newbalanceDest:line[9], dataLine:line[0]}]->(b)

''').data()

# How many entries ?

In [40]:
x = graph.run('''
        MATCH (a:Account)
        RETURN COUNT(a) AS ACCOUNT
    ''').data()[0]

print("Writing %d accounts" % x['ACCOUNT'])

Writing 1800 accounts


In [41]:
x = graph.run('''
        MATCH ()-[r]->()
        RETURN COUNT(r) AS OPERATION
    ''').data()[0]

print("Writing %d operations" % x['OPERATION'])

Writing 571039 operations
